# Test OpenAI Adapter

This notebook tests the OpenAI adapter with different configurations for code-switching analysis.


In [1]:
# Setup: imports and environment
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Add src to path
sys.path.append('../src')

# Load API keys from .env file
load_dotenv()

# Import our adapter
from adapters.openai_adapter import OpenAIAdapter

# Verify that your key loaded correctly
print("OpenAI key:", bool(os.getenv("OPENAI_API_KEY")))


OpenAI key: True


In [2]:
# Initialize OpenAI adapter
openai_adapter = OpenAIAdapter(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",  # Cost-effective model
    temperature=0.7,
    max_tokens=500
)

print("OpenAI adapter initialized successfully!")


OpenAI adapter initialized successfully!


In [3]:
# Test basic generation
test_prompt = "Paraphrase this sentence without changing its meaning: He finna go to the store."

response = openai_adapter.generate_response(test_prompt)
print("Response:", response)


Error generating response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Response: None


In [12]:
# Test with system prompt for better code-switching analysis
system_prompt = """You are a linguist analyzing code-switching behavior. 
Your task is to preserve the dialectal style and cultural context while 
paraphrasing or continuing text. Maintain the same linguistic variety 
and register as the input."""

user_prompt = "Paraphrase this AAVE sentence: He finna go to the store. You sliding?"

response = openai_adapter.generate_with_system_prompt(system_prompt, user_prompt)
print("Response with system prompt:", response)


Error generating response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Response with system prompt: None


In [13]:
# Test multiple candidates
candidates = openai_adapter.generate_multiple_candidates(
    "Continue this Spanglish sentence: Vamos later, it's muy close to la tienda.",
    num_candidates=3
)

print("Multiple candidates:")
for i, candidate in enumerate(candidates, 1):
    print(f"{i}. {candidate}")


Error generating candidates: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Multiple candidates:


In [14]:
# Test different temperature settings
temperatures = [0.2, 0.5, 0.8]
prompt = "Explain this British English phrase: 'We're off on holiday next week, fancy it?'"

print("Testing different temperature settings:")
print("=" * 50)

for temp in temperatures:
    openai_adapter.update_config(temperature=temp)
    response = openai_adapter.generate_response(prompt)
    print(f"\n🌡️ Temperature {temp}:")
    print(f"Response: {response}")
    print("-" * 30)


Testing different temperature settings:
Error generating response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

🌡️ Temperature 0.2:
Response: None
------------------------------
Error generating response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

🌡️ Temperature 0.5:
Response: None
------------------------------
Error generating response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For m

In [15]:
# Test with retry mechanism
print("Testing retry mechanism:")
print("=" * 30)

response = openai_adapter.generate_with_retry(
    "Paraphrase this: Ion think that plan gon' work.",
    max_retries=3
)
print(f"Response with retry: {response}")


Testing retry mechanism:
Attempt 1 failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Attempt 2 failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Attempt 3 failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficie

In [ ]:
# Get available models
print("Available OpenAI models:")
print("=" * 30)

models = openai_adapter.get_available_models()
for model in models:
    print(f"• {model}")

print(f"\nTotal models available: {len(models)}")


In [ ]:
# Test with different models
print("Testing different OpenAI models:")
print("=" * 40)

models_to_test = ["gpt-4o-mini", "gpt-3.5-turbo"]
test_prompt = "Continue this sentence in the same style: We was tryna finish that yesterday"

for model in models_to_test:
    try:
        print(f"\n🤖 Testing {model}:")
        adapter = OpenAIAdapter(
            api_key=os.getenv("OPENAI_API_KEY"),
            model=model,
            temperature=0.7,
            max_tokens=200
        )
        response = adapter.generate_response(test_prompt)
        print(f"Response: {response}")
        print("-" * 40)
    except Exception as e:
        print(f"❌ Error with {model}: {e}")
        print("-" * 40)


In [ ]:
# Test batch processing with our stimuli data
print("Loading stimuli data for batch processing:")
print("=" * 45)

stimuli = pd.read_csv("../data/raw/stimuli.csv")
print(f"Loaded {len(stimuli)} stimuli")
print("\nFirst few examples:")
stimuli.head()


In [ ]:
# Process a few examples with OpenAI
from tqdm import tqdm

print("Processing stimuli with OpenAI:")
print("=" * 35)

# Reset to good settings for batch processing
openai_adapter.update_config(temperature=0.5, max_tokens=300)

responses = []
for i, row in tqdm(stimuli.head(3).iterrows(), total=3):
    # Create task-specific prompt
    if row.task == "paraphrase":
        prompt = f"Paraphrase this text in the same dialectal style: {row.text}"
    elif row.task == "explain":
        prompt = f"Explain this text in simple terms while preserving the dialectal style: {row.text}"
    elif row.task == "continue":
        prompt = f"Continue this text in the same dialectal style: {row.text}"
    else:
        prompt = f"Process this text while maintaining the dialectal style: {row.text}"
    
    output = openai_adapter.generate_response(prompt)
    responses.append({
        "id": row.id,
        "variety": row.variety,
        "task": row.task,
        "input_text": row.text,
        "output_text": output
    })

df = pd.DataFrame(responses)
print("\nOpenAI responses:")
df
